In [1]:
from casatasks import *
# from casatools import *
import casatools
import os

from casaplotms import plotms
from casaviewer.imview import imview
import glob
import pandas as pd
from tqdm import tqdm
msmd = casatools.msmetadata()
ms = casatools.ms()
tb = casatools.table()
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
def reset_rc_params():
    mpl.rcParams.update({'font.size': 14, 
        'mathtext.fontset': 'stix',
        "text.usetex": False,
        "font.family": "sans-serif",
        "font.family": "sans",
        "font.serif": ["Exo 2"],
        "font.sans-serif": ["Exo 2"],
        'font.family': 'STIXGeneral', 
        'xtick.labelsize':16,
        'ytick.labelsize':16,
        'axes.labelsize' : 16,
        'xtick.major.width':1,
        'ytick.major.width':1,
        'axes.linewidth':1,
        'lines.linewidth':2,
        'legend.fontsize':14,
        "grid.linestyle":'--',                
        })
    pass
reset_rc_params()

def report_flag(summary,axis):
    for id, stats in summary[ axis ].items():
        print('%s %s: %5.1f percent flagged' % ( axis, id, 100. * stats[ 'flagged' ] / stats[ 'total' ] ))
    pass


This notebook contain a series of commands to help with data combination. <br>
It works with measurement sets from the same instrument or from different instruments. <br>
Though, it was tested only with e-MERLIN and VLA. <br>

What it does:
- From a reference measurement set, take the phase centre from that and uses to shift the phasecentre according to it
- Calculate the statistical weights and produce a combined visibility with homogeneous weights. Scalling factors ara calculatet using `statwt`.

***This is useful when combining observations of different epochs that have a slightly different phase pointing centre, and also different statistical weights.***

# Example with Ku band JVLA
More examples will be added here, on a source-per-source basis.

## Ku Band VLA

In [2]:
ref_vis = '/media/sagauga/starbyte/LIRGI_Sample/VLA-Archive/A_config/Ku_band/Arp299/22A-314.sb41475013.eb41775491.59690.14839207176/fields/Arp299/Arp299.calibrated.ms'

In [3]:
ref_info = listobs(vis=ref_vis,listfile=ref_vis+'.listobs',overwrite=True)

In [4]:
import subprocess
import re

In [5]:
file_obs = ref_vis+'.listobs'

In [ ]:
def get_phase_centre(vis):
    
    from astropy.coordinates import SkyCoord
    import astropy.units as u

    msmd.open(vis)
    ra_radians = msmd.phasecenter()['m0']['value']
    dec_radians = msmd.phasecenter()['m1']['value']
    msmd.close()
    # Convert to SkyCoord object
    coord = SkyCoord(ra=ra_radians*u.radian, dec=dec_radians*u.radian, frame='icrs')

    # Format the output using 'hmsdms'
    formatted_coord = coord.to_string('hmsdms')
    formatted_ra, formatted_dec = formatted_coord.split()
    
    formatted_ra_hms = formatted_ra.replace('h', ':').replace('m', ':').replace('s', '')
    formatted_dec_dms = formatted_dec.replace('d', '.').replace('m', '.').replace('s', '')

    formatted_output = "J2000 {} {}".format(formatted_ra_hms, formatted_dec_dms)

    print(formatted_output)
    return(formatted_output)


In [7]:
get_phase_centre(ref_vis)

J2000 11:28:31.320000 +58.33.41.69999


'J2000 11:28:31.320000 +58.33.41.69999'

In [8]:
# os.system('cat '+ref_vis+'.listobs')

In [9]:
# ref_phasecentre = 'J2000 11:28:31.320000 +58.33.41.69999'
ref_phasecentre = get_phase_centre(ref_vis)

J2000 11:28:31.320000 +58.33.41.69999


In [9]:
# ref_phasecentre = 'J2000 11:28:31.320000 +58.33.41.69999'
names = ['/media/sagauga/starbyte/LIRGI_Sample/VLA-Archive/A_config/Ku_band/Arp299/20B-279.sb39070014.eb39191064.59204.52319636574/fields/Arp299/Arp299.calibrated.ms']
# Shift the phasecentre
for name in names:
    phaseshift(vis=name,phasecenter=ref_phasecentre,
               outputvis=name.replace('.ms','')+'_phaseshift.ms')

In [10]:
names = ['/media/sagauga/starbyte/LIRGI_Sample/VLA-Archive/A_config/Ku_band/Arp299/20B-279.sb39070014.eb39191064.59204.52319636574/fields/Arp299/Arp299.calibrated_phaseshift.ms',
         ref_vis
         ]

In [11]:
names

['/media/sagauga/starbyte/LIRGI_Sample/VLA-Archive/A_config/Ku_band/Arp299/20B-279.sb39070014.eb39191064.59204.52319636574/fields/Arp299/Arp299.calibrated_phaseshift.ms',
 '/media/sagauga/starbyte/LIRGI_Sample/VLA-Archive/A_config/Ku_band/Arp299/22A-314.sb41475013.eb41775491.59690.14839207176/fields/Arp299/Arp299.calibrated.ms']

In [12]:
list_obs_info = []
for name in tqdm(names):
    vis = name + ''
    list_obs_info_i = listobs(vis=vis,listfile=name+'.listobs',overwrite=True)
    list_obs_info.append(list_obs_info_i)
df_lo = pd.DataFrame(list_obs_info)

for i in range(len(df_lo['field_0'])):
    print(df_lo['field_0'][i]['name'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.18it/s]

SN 2010P
NGC3690


In [13]:
stats = []
for name in tqdm(names):
    vis = name + ''
    stats_i = statwt(vis=vis,preview=False,datacolumn='data',timebin='12s',statalg='chauvenet')
    stats.append(stats_i)
df = pd.DataFrame(stats)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [07:25<00:00, 222.64s/it]


In [15]:
wt_mean = np.mean(df['mean'])
df['mean']
df['wt_factor'] = wt_mean/df['mean']
df['wt_factor']

0    0.973815
1    1.027632
Name: wt_factor, dtype: float64

In [16]:
concatvis='/media/sagauga/starbyte/LIRGI_Sample/VLA-Archive/A_config/Ku_band/Arp299/concatenated_calibrated_data/Arp299_multi_2x_Ku_EVLA.ms'
concat(vis=names,
       concatvis=concatvis,
       freqtol='1MHz',visweightscale=list(df['wt_factor'])
      )

In [17]:
split(vis=concatvis,
      outputvis=concatvis.replace('.ms','.avg6s.ms'),
      datacolumn='data',timebin='6s')